In [1]:
import os
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import cv2
import keras
from tensorflow.keras.optimizers import RMSprop, SGD
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D,BatchNormalization, MaxPooling2D
from keras.preprocessing.image import img_to_array, load_img


In [2]:
# Create Dataset

def create_data(data_dir,class_folder):
  directory = data_dir + "/" + class_folder + "/"
  data = os.listdir(directory)
  random.shuffle(data)
  train = data[:100]
  test = data[100:]

  return train,test

class_folder = []
path = "/content/drive/MyDrive/Dataset/15-Scene/15-Scene"

for category in sorted(os.listdir(path)):
  class_folder.append(category)

train_set = []
test_set = []

data_dir = '/content/drive/MyDrive/Dataset/15-Scene/15-Scene'

for class_name in class_folder:
  train,test = create_data(data_dir,class_name)
  train_data = [class_name,train]
  test_data = [class_name,test]
  train_set.append(train_data)
  test_set.append(test_data)

train_set = np.array(train_set,dtype=object)
test_set = np.array(test_set,dtype=object)

In [ ]:
# Create directory to store train and test folder

create_dir = "/content/drive/MyDrive/Dataset/Train_Test"
output_directory = os.mkdir(os.path.join(create_dir,'Scene15_Output'))

In [ ]:
def write_img_to_folder(data_dir,output_path,output_directory,folder_name,dataset):

  #new_folder = os.mkdir(os.path.join(output_directory,folder_name))

  for data in dataset:
    class_name = data[0]
    data_files = data[1]

    data_path = os.path.join(output_path + "/" + str(folder_name) ,class_name)
    #category_dire = os.mkdir(data_path)

    for filename in data_files:
      img = cv2.imread(data_dir + "/" + class_name + "/" + filename, 0)
      #cv2.imwrite(os.path.join(data_path, filename), img)

output_path = "/content/drive/MyDrive/Dataset/Train_Test/Scene15_Output"
output_directory = "/content/drive/MyDrive/Dataset/Train_Test/Scene15_Output"

# Train/Test folder using write_img_to_folder function

train_folder = write_img_to_folder(data_dir,output_path,output_directory,'Train', train_set)
test_folder = write_img_to_folder(data_dir, output_path,output_directory,'Test', test_set)

In [3]:
def convert_img_to_array(dir_path):
    X = []
    y = []
    one_hot_lookups = np.eye(15)

    for category in sorted(os.listdir(dir_path)):

      for files in os.listdir(dir_path + category):

        img = load_img(dir_path + category + '/' + files,target_size=(224, 224))
        #img = cv2.resize(img, dsize = (227,227))
        img = img_to_array(img) 
        img = tf.image.per_image_standardization(img)  
        X.append(img) 
        y.append(np.reshape(one_hot_lookups[int(category)],[15]))  

    return X,y

In [4]:
def load_data():
    train_dir_path = "/content/drive/MyDrive/Dataset/Train_Test/Scene15_Output/Train/"
    X_train,y_train = convert_img_to_array(train_dir_path)
    
    test_dir_path = "/content/drive/MyDrive/Dataset/Train_Test/Scene15_Output/Test/"
    X_test, y_test = convert_img_to_array(test_dir_path)
    
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    
    X_train /= np.std(X_train, axis = 0)
    X_test /= np.std(X_test, axis = 0)
    
    return ((X_train, y_train), (X_test, y_test))
(x_train, y_train), (x_test, y_test) = load_data()


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1500, 224, 224, 3)
(1500, 15)
(2985, 224, 224, 3)
(2985, 15)


In [5]:
model = Sequential()
# 1st layer..........
model.add(Conv2D(96, kernel_size = (11, 11), strides = (4,4), activation='relu', padding = 'valid' , data_format='channels_last', input_shape=(224, 224, 3),name = "first"))
model.add(MaxPooling2D(pool_size=(3,3) , strides = (2,2) , padding = 'valid' )) 
model.add(BatchNormalization())

# 2nd layer................
model.add(Conv2D(256, kernel_size=(5,5), activation='relu', strides=(1,1), padding = 'valid', name = "second"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

#3rd layer.................
model.add(Conv2D(384, kernel_size=(3,3), activation='relu', strides=(1,1), padding='valid', name = "third"))
model.add(BatchNormalization())

#4th layer.................
model.add(Conv2D(384, kernel_size=(3,3), activation='relu', strides=(1,1), padding='valid', name = "fourth"))
model.add(BatchNormalization())

#5th layer.................
model.add(Conv2D(256, kernel_size=(3,3), strides=(1,1), activation = 'relu', padding='valid' , name = "fifth"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

model.add(Dropout(0.2))

model.add(Flatten())

# Fully Connected Layer 1
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))

# Fully Connected Layer 2
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(15, activation = 'softmax')) #As number of nodes in last layer in softmax is number of classes, where each node is probability of classes

In [6]:
# Compile the model
optimizer = SGD(learning_rate = 1e-3,momentum = 0.9)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(x_train, y_train, batch_size=5, epochs=50, verbose=1, shuffle=True,validation_data=(x_test,y_test))

Epoch 1/50
300/300 [==============================] - 52s 73ms/step - loss: 3.2206 - accuracy: 0.1267 - val_loss: 2.7656 - val_accuracy: 0.1484
Epoch 2/50
300/300 [==============================] - 16s 55ms/step - loss: 2.5924 - accuracy: 0.1720 - val_loss: 2.6208 - val_accuracy: 0.1930
Epoch 3/50
300/300 [==============================] - 17s 56ms/step - loss: 2.5464 - accuracy: 0.1760 - val_loss: 2.8282 - val_accuracy: 0.1487
Epoch 4/50
300/300 [==============================] - 17s 56ms/step - loss: 2.5234 - accuracy: 0.1953 - val_loss: 2.3368 - val_accuracy: 0.2606
Epoch 5/50
300/300 [==============================] - 17s 56ms/step - loss: 2.4702 - accuracy: 0.2047 - val_loss: 2.5555 - val_accuracy: 0.2168
Epoch 6/50
300/300 [==============================] - 21s 69ms/step - loss: 2.5694 - accuracy: 0.2073 - val_loss: 2.4166 - val_accuracy: 0.2231
Epoch 7/50
300/300 [==============================] - 21s 69ms/step - loss: 2.6438 - accuracy: 0.2140 - val_loss: 2.5528 - val_accuracy: